In [50]:
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore
from surprise.model_selection import train_test_split
from surprise import accuracy, Prediction
import surprise

# 加载数据集
# data = Dataset.load_builtin('ml-100k')
reader = Reader(line_format='user item rating', sep=',', rating_scale=(1.0, 5.0))
data = Dataset.load_from_file('./ml-latest-small/ratings.csv', reader)

# 创建协同过滤算法模型
sim_options = {'name': 'pearson', 'user_based': True}  # 使用余弦相似度进行用户间的相似度计算
model = KNNWithZScore(sim_options=sim_options)

# 划分数据集为训练集和测试集
trainset, testset = train_test_split(data, train_size=0.7, test_size=0.3, shuffle=True)

In [51]:
import numpy as np
# 训练模型
model.fit(trainset)
# 预测
predictions = model.test(testset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [52]:
# 评估模型
accuracy.rmse(predictions)
accuracy.mae(predictions)
print(len(predictions), len(testset))


RMSE: 0.8971
MAE:  0.6793
30251 30251


In [53]:
all_precision = 0
all_recall = 0
all_mse = 0
all_mae = 0
cnt = 0
from tqdm import tqdm
for user_id in tqdm(range(1, 611)):  # 目标用户ID
    user_id = str(user_id)
    
    testset_user_id_movies = []
    for item in predictions:
        if item.uid == user_id:
            testset_user_id_movies.append(item)
    # print(testset_user_id_movies)
    
    items_to_recommend = 10  # 推荐物品数量
        
    # 按实际值排序
    movies_real = testset_user_id_movies.copy()
    movies_real = sorted(movies_real, key=lambda x: x.r_ui, reverse=False)
    movies_real_id = [int(item.iid) for item in movies_real]
    movies_real_id = sorted(movies_real_id)
    # print(movies_real_id)
    
    # 按估计值排序
    movies_est = testset_user_id_movies.copy()
    movies_est = sorted(movies_est, key=lambda x: x.est, reverse=False)
    movies_est_id = [int(item.iid) for item in movies_est[:items_to_recommend]]
    movies_est_id = sorted(movies_est_id)
    # print(movies_est_id)
    all_mse += sum([(item.r_ui - item.est)**2 for item in movies_est[:items_to_recommend]])
    all_mae += sum([abs(item.r_ui - item.est) for item in movies_est[:items_to_recommend]])
    
    # 计算相同数量
    hit = 0
    index_movies_est_id = 0
    index_movies_real_id = 0
    length = min(len(movies_est_id), len(movies_real_id))
    cnt += length
    while index_movies_est_id != length and index_movies_real_id != length:
        if movies_est_id[index_movies_est_id] == movies_real_id[index_movies_real_id]:
            hit += 1
            index_movies_est_id += 1
            index_movies_real_id += 1
        elif movies_est_id[index_movies_est_id] < movies_real_id[index_movies_real_id]:
            index_movies_est_id += 1
        else:
            index_movies_real_id += 1
    # print(hit)
    
    precision = hit / len(movies_est_id)
    recall = hit / len(movies_real_id)
    all_precision += precision
    all_recall += recall
    # print(f"{user_id} precision = {precision:3f}, recall = {recall:3f}")

100%|██████████| 610/610 [00:01<00:00, 402.52it/s]


In [54]:
print("precision: ", all_precision / 610)
print("recall: ", all_recall / 610)
print("rmse: ", np.sqrt(all_mse))
print("mae: ", all_mae / cnt)

precision:  0.5040983606557379
recall:  0.38603086909970175
rmse:  75.16086981989127
mae:  0.7648385652522741
